# How to Choose the Best Nearest Neighbors Algorithm?
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/stephenleo/adventures-with-ann/blob/main/ann_benchmarking.ipynb)

A data-driven approach to choose the fastest and most accurate ANN algorithm on your custom dataset

![Header Image](images/ann_benchmarks_header.jpeg)

# ANN Background
In my previous post [KNN is Dead!](https://medium.com/towards-artificial-intelligence/knn-k-nearest-neighbors-is-dead-fc16507eb3e), I have compared an ANN algorithm called `HNSW` with `sklearn`'s KNN and proved that HNSW has vastly superior performance with a 380X speed up while delivering 99.3% of the same results. 

To make things even more interesting, there are several ANN algorithms such as 
1. Spotify's [ANNOY](https://github.com/spotify/annoy)
2. Google's [ScaNN](https://github.com/google-research/google-research/tree/master/scann)
3. Facebook's [Faiss](https://github.com/facebookresearch/faiss)
4. My personal favorite: Hierarchical Navigable Small World graphs [HNSW](https://github.com/nmslib/hnswlib)
5. and many more

As a data scientist, I am a huge proponent of making data-driven decisions, as I mentioned in [How to Choose the Best Keras Pre-Trained Model](https://towardsdatascience.com/how-to-choose-the-best-keras-pre-trained-model-for-image-classification-b850ca4428d4). So, in this post, I'll demonstrate a data-driven way to decide which ANN algorithm is the best choice for your custom dataset by using the excellent `ann-benchmarks` [GitHub repository](https://github.com/erikbern/ann-benchmarks).

# ANN Benchmarks
The ann-benchmarks code compares multiple ANN algorithms by plotting each algorithm's Recall vs Queries per second. In this way, you can visually choose the fastest (`Queries per second`) algorithm while delivering a minimum desired `Recall`. We choose Recall as the metric of interest because we want to know what percent of true Nearest Neighbors are retrieved by the ANN [Precision and Recall](https://en.wikipedia.org/wiki/Precision_and_recall).

![Precision_Recall](images/precision_recall.png)

Below image is the graph resulting from running ANN-Benchmarks on glove-100 dataset using angular distance metric. On this dataset, the scann algorithm has the highest Queries per second at any given Recall and thus the best algorithm on this dataset.

![Glove-100-Angular](images/glove-100-angular.png)

# Setup
These are the steps involved in running the ann-benchmarks code on your custom dataset. 
1. Install ann-benchmarks in a python 3.6 environment.
2. Upload your Custom Embeddings DataFrame into ann-benchmarks/data directory.
3. Update ann-benchmarks/ann-benchmarks/dataset.py to read and split your new Custom Embeddings DataFrame.
4. Run the benchmarking code.
5. Plot the results.

# Install ann-benchmarks in a python 3.6 environment
The code for this step needs to be executed in your `Terminal`. I'm using anaconda for the environment setup. This will take several minutes to complete, which can be sped up by increasing the number of concurrent processes using the `--proc` parameter. I intentionally upgrade `pandas` and `scipy` only after installation is completed.

**_Important!_** Ann-benchmarks only support Python 3.6 as of this writing.

```
conda create -n ann python=3.6 jupyterlab -y
conda activate ann
git clone https://github.com/erikbern/ann-benchmarks.git
cd ann-benchmarks/
pip install -r requirements.txt
python install.py --proc=8
pip install --upgrade pandas scipy
mkdir data
```

## Potential Issues:
1. `gcc` is not installed: Install `gcc` using `sudo apt-get install gcc`.
2. `Docker` is not installed: Install `Docker` following instructions from this [Link](https://docs.docker.com/engine/install/ubuntu/). Just remember to select your correct distro.
3. Permission issues: If you encounter any permission issues running `python install.py`, just run it with `sudo /opt/conda/envs/ann/bin/python install.py`. Remember to provide the full path to anaconda python in your `ann` environment when you use `sudo`.

# Upload your Custom Embeddings DataFrame
In this step, you would paste your custom data frame file into the `ann-benchmarks/data directory`. For this post, my data frame is the same [Amazon product dataset](http://deepyeti.ucsd.edu/jianmo/amazon/) with `FastText` sentence embedding that I used in my post: [KNN (K-Nearest Neighbors) is Dead!](https://medium.com/towards-artificial-intelligence/knn-k-nearest-neighbors-is-dead-fc16507eb3e) But I'm only sampling a random 50K rows to ensure the benchmarking runs in a reasonable time. Below is the code to save the embedding data frame as a pickle file called `custom-euclidean.pkl` in the correct directory and also an excerpt of the first 5 rows in the data frame.

In [2]:
import pandas as pd

# Read your features/embedding dataframe
df = pd.read_pickle('df.pkl')

# Random sample 50K rows to keep run time reasonable
df = df.sample(50000)
df.reset_index(inplace=True, drop=True)

df.to_pickle('ann-benchmarks/data/custom-euclidean.pkl')

print(df.shape)
df.head()

(50000, 2)


,title,emb
0,4 items Combo: ITUFFY(TM) Mini Stylus Pen + LC...,"[0.003052382, 0.024353715, 0.0337113, -0.00305..."
1,Shell Case / Belt Clip Holster Combo for HTC E...,"[0.03329156, 0.008657752, 0.010499144, -0.0037..."
2,CELL PHONE CASE COVER FOR SAMSUNG INTENSITY II...,"[0.026096184, -0.043489173, 0.030792506, -0.03..."
3,"HTC Desire 530 Case, LUXCA Designed Armor [Sho...","[0.0129606975, 0.006081905, 0.045901332, 0.004..."
4,Importer520 Luxury Luxurious Synthetic Leather...,"[0.041510176, 0.018833052, 0.03515195, 0.01226..."


# Update datasets.py to handle your Custom DataFrame
We need to update the ANN Benchmarks code to know how to handle our new data frame. We do that by adding a new function and dictionary element at the end of the `ann-benchmarks/ann-benchmarks/datasets.py` file. The allowed options for distance argument are `"euclidean"`, `"angular"`, `"hamming"` or `"jaccard"`. The choice of distance metric is specific to your problem. For my case, I found that `"euclidean"` provided good nearest neighbors.

In [ ]:
# Add this code to the end of ann-benchmarks/ann-benchmarks/datasets.py
def custom_dataset(out_fn, test_ratio, distance):
    # Function to handle our custom dataset
    
    import pandas as pd
    
    # Read the Data Frame
    # out_fn is of the form 'data/<dataset-name>.hdf5'
    df = pd.read_pickle(out_fn.split('.')[0]+'.pkl')
    
    # Convert single embedding column to numpy list of lists
    X = pd.DataFrame(df['emb'].tolist()).to_numpy()
    # Split Train and Test
    X_train, X_test = train_test_split(X, test_size=test_ratio)
    
    # Write HDF5 Output
    write_output(X_train, X_test, out_fn, distance)
    
# Create a new dictionary element to call our new function
# 20% of rows used as Test Set
# Euclidean distance used as measure for finding neighbors
DATASETS['custom-euclidean'] = lambda out_fn: custom_dataset(out_fn, test_ratio=0.2, distance='euclidean')

# Run the benchmarking code
If everything was smooth up to this point, we could now run the benchmarking simply by calling the below line from your `Terminal`. Change the value of `parallelism` to as many CPU cores you want to use. I'm using a 16 core CPU, so I choose `parallelism=14` to reserve 2 cores for other tasks. This will take some time to complete. My 50K rows data frame with a 20% Test set ran for ~7 hours.

```
python run.py --dataset='custom-euclidean' --parallelism=14
```

# Plot the results
Once the run has complete, we can plot the results by running `plot.py`. We can also enable the y-axis to plot in `log` scale. Notice here that I am using `sudo` with the full path to my Anaconda Python because I encountered permission issues while trying to run plot.py normally: `python plot.py --dataset=custom-euclidean --y-log`. You can use whichever works for you.

```
sudo /opt/conda/envs/ann/bin/python plot.py --dataset=custom-euclidean --y-log
```

The resultant plot is saved in the results directory as a png file. For the 50K rows Amazon dataset that I've used in this post, the result is below.

![Custom-Amazon-Products-Dataset-Euclidean](images/custom-euclidean.png)

From this plot, we see that a few algorithms, such as `NGT-onng`, `hnsw(nmslib)`, `n2`, `hnswlib`, `SW-graph(nmslib)` are clearly better than the rest, by delivering higher `Queries per second` at any given `Recall`. Thus, we can further explore these algorithms for our project on Amazon product dataset.

# Conclusion
In conclusion, by using `ann-benchmarks`, and putting together some additional code, we can test a large number of ANN algorithms on our own custom dataset to narrow down just a few ANN algorithms for further exploration. 

All the code for this post can be found on my [Github repository](https://github.com/stephenleo/adventures-with-ann/blob/main/ann_benchmarking.ipynb). Thank you for reading!